# A Recommendation Engine Using Python for An Episode Available on Spotify Podcast

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas
import numpy as np
import datetime as dt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

from ipynb.fs.full.Credentials import *

## Data Collection

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id = Spotify_Client_ID, 
                                                      client_secret = Spotify_Client_Secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
episode_results = sp.search(q = 'data',
                            type = 'episode',
                            market = 'US',
                            limit = 1,
                            offset = 0
                           )
print(json.dumps(episode_results, indent=2))

{
  "episodes": {
    "href": "https://api.spotify.com/v1/search?query=data&type=episode&market=US&offset=0&limit=1",
    "items": [
      {
        "audio_preview_url": "https://p.scdn.co/mp3-preview/bd47b762e2cfbe424249e440ed041fc870b0dae6",
        "description": "Download for Mobile\u00a0|\u00a0Podcast Preview |  Full Timestamps Hyper Dragonball Z, The End of Vagabond, $GME, RE:VERSE, Copyright Claiming Cosplay, Toxic Stat Shaming. You can watch us record the podcast live on twitch.tv/castlesuperbeast Outro: Daredevil - Ace Combat 7 Download the DoorDash app in the App Store and enter code CASTLE2021 to get 25% off and zero delivery fees on your first order! --Go to http://imperfectfoods.com and use code SUPERBEAST to get 20% off PLUS free shipping!   Gamestop stock up 125% thanks to r/wallstreetbets  Japanese government discussing stricter copyright laws on cosplay, top cosplayer Enako chimes in  Report: Tencent raising billions to buy EA, Take-Two, or others  Konami Restructuring

In [4]:
def getEpisodes(queries, total):
    name = []
    description = []
    duration = []
    explicit = []
    language = []
    release_date = []
    url = []

    if total <= 50:
        limit = total
    else:
        limit = 50

    for query in queries:
        print(query)
        for i in range(0, total, limit):
            results = sp.search(q = query,
                                type = 'episode',
                                market = 'US',
                                limit = limit,
                                offset = i
                               )
            
            for i, t in enumerate(results['episodes']['items']):
                name.append(t['name'])
                description.append(t['description'])
                duration.append(t['duration_ms'])
                explicit.append(t['explicit'])
                language.append(t['language'])
                release_date.append(t['release_date'])
                url.append(t['external_urls']['spotify'])

    pd.set_option('display.max_colwidth', 0)

    dataframe = pd.DataFrame({'Name' : name,
                              'Description' : description,
                              'Explicit' : explicit,
                              'Language' : language,
                              'Duration (ms)' : duration,
                              'Release Date' : release_date,
                              'URL' : url,
                             }).drop_duplicates().reset_index(drop = True)
    
    return dataframe

In [5]:
episodes_data = getEpisodes(['h', 'b', 'f', 'm'], 1000)
print(episodes_data.shape)
episodes_data

h
b
f
m
(3361, 7)


,Name,Description,Explicit,Language,Duration (ms),Release Date,URL
0,Introducing: You Heard Me Write,"What happens when 30 artists harness the power of creativity to form meaningful connections? Here’s a sneak peak, for your listening pleasure!",False,en,88764,2021-02-11,https://open.spotify.com/episode/5l8afcnfvY3xeP33XKzMtd
1,A Broken System for Housing the Homeless,"This episode contains descriptions of sexual violence. Victor Rivera has framed his life story as one of redemption and salvation. Escaping homelessness and drug addiction, he founded the Bronx Parent Housing Network, one of the largest nonprofits operating homeless shelters in New York City.But that’s not the whole story. A Times investigation has found a pattern of allegations of sexual abuse and financial misconduct against him during his career.We look at the accusations against Mr. Rivera and ask what lessons can be learned.Guest: Amy Julia Harris, an investigative reporter on The New York Times’s Metro desk. For an exclusive look at how the biggest stories on our show come together, subscribe to our newsletter. You can read the latest edition here.Background reading: Victor Rivera gained power and profit as New York’s homeless crisis worsened.After the Times investigation, Mr. Rivera was fired and now faces a criminal inquiry.For more information on today’s episode, visit nytimes.com/thedaily. Transcripts of each episode will be made available by the next workday.",True,en,1859709,2021-02-11,https://open.spotify.com/episode/7BHkqvyD4GvWubcMiYNGrJ
2,1 Hour Sleep Clinical Hypnosis Healing Sleep Music,"Please enjoy this 1 hour Relaxing Music for Stress Relief. Soothing Music. Meditation. Sleep Music. This audio will create a relaxing background atmosphere to convey peace and relaxation. It is an excellent for Relaxation, Meditation, Concentration and Balance, music therapy. This peaceful relaxing composition can be used as Deep Meditation Music, Music for Yoga and Pilates , Music for Massage , Spa Music. Also this music is perfect as dream music, Healing music, Study Music, Sleep Music to beat insomnia and the Music for complete Relaxation DISCLAIMER: Do not listen to this recording while driving or operating machinery. Hypnosis can be used as a complementary treatment to traditional medicine and not instead. If you suffer from a medical condition please consult with your doctor. Hypnosis is not a replacement for medical treatment. Always seek the advice of your physician or other qualified mental health provider with any questions you may have regarding a medical condition or mental disorder. Never disregard professional medical advice or delay in seeking it because of something you have read on our channel. Nothing found on our channel is intended to be a substitute for professional psychological, psychiatric or medical advice, diagnosis, or treatment. This recording is in no way a replacement for any prescribed medication; nor is it intended to contraindicate or supersede any medically diagnosed conditions. Audice is a Certified Clinical Hypnotist and Board licensed Neuro-linguistic Programming Practitioner and TimeLine Therapy® Practitioner, registered with ABH-NLP –American Board for Hypnotherapy and NLP and TimeLine Therapy® Association. She is a registered member with NATIONAL GUILD OF HYPNOTISTS. Visit https://audicewellnessservices.com/ Please LINK & SHARE this audio with those who you think might benefit from positive hypnosis and guided meditation. Thank you. --- Send in a voice message: https://anchor.fm/audice-wellness-services/message",False,en,3609052,2020-03-22,https://open.spotify.com/episode/6RCGwAKKCXF3WxmPZ9KYTo
3,"When Covid Hit Nursing Homes, Part 1: ‘My Mother Died Alone’","When New York was the epicenter of the pandemic in the United States, Gov. Andrew Cuomo emerged as a singular, strong leader. Now his leadership is embattled, particularly over the extent of deaths in nursing homes during the peak.Today, in the first of two parts on 

## Exploratory Data Analysis

In [6]:
episodes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3361 entries, 0 to 3360
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           3361 non-null   object
 1   Description    3361 non-null   object
 2   Explicit       3361 non-null   bool  
 3   Language       3361 non-null   object
 4   Duration (ms)  3361 non-null   int64 
 5   Release Date   3361 non-null   object
 6   URL            3361 non-null   object
dtypes: bool(1), int64(1), object(5)
memory usage: 161.0+ KB


In [7]:
episodes_data['Days ago'] = (np.datetime64(dt.date.today()) - episodes_data['Release Date'].values.astype('datetime64[D]')).astype(int)
episodes_data['Name + Description'] = episodes_data['Name'] + ' ' + episodes_data['Description']
episodes_data

,Name,Description,Explicit,Language,Duration (ms),Release Date,URL,Days ago,Name + Description
0,Introducing: You Heard Me Write,"What happens when 30 artists harness the power of creativity to form meaningful connections? Here’s a sneak peak, for your listening pleasure!",False,en,88764,2021-02-11,https://open.spotify.com/episode/5l8afcnfvY3xeP33XKzMtd,19,"Introducing: You Heard Me Write What happens when 30 artists harness the power of creativity to form meaningful connections? Here’s a sneak peak, for your listening pleasure!"
1,A Broken System for Housing the Homeless,"This episode contains descriptions of sexual violence. Victor Rivera has framed his life story as one of redemption and salvation. Escaping homelessness and drug addiction, he founded the Bronx Parent Housing Network, one of the largest nonprofits operating homeless shelters in New York City.But that’s not the whole story. A Times investigation has found a pattern of allegations of sexual abuse and financial misconduct against him during his career.We look at the accusations against Mr. Rivera and ask what lessons can be learned.Guest: Amy Julia Harris, an investigative reporter on The New York Times’s Metro desk. For an exclusive look at how the biggest stories on our show come together, subscribe to our newsletter. You can read the latest edition here.Background reading: Victor Rivera gained power and profit as New York’s homeless crisis worsened.After the Times investigation, Mr. Rivera was fired and now faces a criminal inquiry.For more information on today’s episode, visit nytimes.com/thedaily. Transcripts of each episode will be made available by the next workday.",True,en,1859709,2021-02-11,https://open.spotify.com/episode/7BHkqvyD4GvWubcMiYNGrJ,19,"A Broken System for Housing the Homeless This episode contains descriptions of sexual violence. Victor Rivera has framed his life story as one of redemption and salvation. Escaping homelessness and drug addiction, he founded the Bronx Parent Housing Network, one of the largest nonprofits operating homeless shelters in New York City.But that’s not the whole story. A Times investigation has found a pattern of allegations of sexual abuse and financial misconduct against him during his career.We look at the accusations against Mr. Rivera and ask what lessons can be learned.Guest: Amy Julia Harris, an investigative reporter on The New York Times’s Metro desk. For an exclusive look at how the biggest stories on our show come together, subscribe to our newsletter. You can read the latest edition here.Background reading: Victor Rivera gained power and profit as New York’s homeless crisis worsened.After the Times investigation, Mr. Rivera was fired and now faces a criminal inquiry.For more information on today’s episode, visit nytimes.com/thedaily. Transcripts of each episode will be made available by the next workday."
2,1 Hour Sleep Clinical Hypnosis Healing Sleep Music,"Please enjoy this 1 hour Relaxing Music for Stress Relief. Soothing Music. Meditation. Sleep Music. This audio will create a relaxing background atmosphere to convey peace and relaxation. It is an excellent for Relaxation, Meditation, Concentration and Balance, music therapy. This peaceful relaxing composition can be used as Deep Meditation Music, Music for Yoga and Pilates , Music for Massage , Spa Music. Also this music is perfect as dream music, Healing music, Study Music, Sleep Music to beat insomnia and the Music for complete Relaxation DISCLAIMER: Do not listen to this recording while driving or operating machinery. Hypnosis can be used as a complementary treatment to traditional medicine and not instead. If you suffer from a medical condition please consult with your doctor. Hypnosis is not a replacement for medical treatment. Always seek the advice of your physician or other qualified mental health provider with any questions you may have regarding a medical condition or mental disorder. Never disregard professional medical advice or dela

## Feature Extraction

In [ ]:
column_trans = ColumnTransformer([('exp_category', OneHotEncoder(dtype='int'), ['Explicit']),
                                  ('lang_category', OneHotEncoder(dtype='int'), ['Language']),
                                  ('text', TfidfVectorizer(use_idf = True, stop_words = 'english'), 'Name + Description'),
                                  ('Duration_int', MinMaxScaler(),['Duration (ms)']),
                                  ('Days_ago_int', MinMaxScaler(),['Days ago']),
    
                                 ],
                                 transformer_weights={'exp_category': 1.0,
                                                        'lang_category': 1.0,
                                                        'text': 1.0,
                                                        'Duration_int': 0.5,
                                                        'Days_ago_int': 0.5
                                                       },
                                 remainder='drop')

token = column_trans.fit_transform(episodes_data).toarray()
token

## Cosine Similarity

In [9]:
cos_sim = cosine_similarity(token)
cos_sim

C:\Users\Hwa-WeiWu\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


array([[1.00000000e+00, 3.36772905e-01, 6.66343855e-01, ...,
        3.32407212e-01, 3.33312062e-01, 6.59075131e-05],
       [3.36772905e-01, 1.00000000e+00, 3.36994696e-01, ...,
        6.69317455e-01, 4.23717742e-05, 3.36059725e-01],
       [6.66343855e-01, 3.36994696e-01, 1.00000000e+00, ...,
        3.36622944e-01, 3.33467701e-01, 1.95393341e-03],
       ...,
       [3.32407212e-01, 6.69317455e-01, 3.36622944e-01, ...,
        1.00000000e+00, 1.18643817e-03, 3.40161048e-01],
       [3.33312062e-01, 4.23717742e-05, 3.33467701e-01, ...,
        1.18643817e-03, 1.00000000e+00, 8.24742625e-04],
       [6.59075131e-05, 3.36059725e-01, 1.95393341e-03, ...,
        3.40161048e-01, 8.24742625e-04, 1.00000000e+00]])

In [10]:
cos_sim.shape

(3361, 3361)

In [11]:
list(episodes_data['Name'])

['Introducing: You Heard Me Write',
 'A Broken System for Housing the Homeless',
 '1 Hour Sleep Clinical Hypnosis Healing Sleep Music ',
 'When Covid Hit Nursing Homes, Part 1: ‘My Mother Died Alone’',
 'When Covid Hit Nursing Homes, Part 2: ‘They’re Not Giving Us an Ending’',
 '#1593 - Dr. Carl Hart',
 'MYSTERIOUS DEATH OF: Holly Bartlett',
 'Hour 2: Heat Homer Detonation',
 'How Blue Holes Work',
 'Death of a Hometown Hero',
 '7X NBA Champion Robert Horry, Coach K Losing His Program And Guys On Chicks',
 'Hell! Hell! Hell!',
 '#1586 - Tony Hinchcliffe',
 'Brené with Jennifer Rudolph Walsh and Ashley C. Ford on Hungry Hearts',
 'How Housing Discrimination Works',
 'MURDERED: Harvey and Jeannette Crewe',
 'I HAVE A BOYFRIEND',
 'Relaxing Rain Sounds for Deep Sleep | 2 Hours',
 'JRE MMA Show #103 with Max Holloway',
 'How Hydropower Works ',
 'Troopz Talking Soccer, This Is March, CBB And Drew Brees is Holding New Orleans Hostage',
 'Ask Him About His Credit Score (w/ Tiffany Haddish)',

## Get Recommendation

In [16]:
def getRecommendation(name, number):
    index = episodes_data[episodes_data['Name'] == name].index.values[0]
    scores = list(enumerate(cos_sim[index]))
    sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)

    n = 0
    print('The ' + str(number) + ' most recommended episodes to ' + name + ' are:\n')
    for index, score in sorted_scores[1:]:
        name = episodes_data[episodes_data.index == index]['Name'].values[0]
        url = episodes_data[episodes_data.index == index]['URL'].values[0]
        print(n + 1, index, name, '\n', url)
        print()
        n += 1
        if n > (number - 1):
            break

In [11]:
list(episodes_data['Name'])

['Introducing: You Heard Me Write',
 'A Broken System for Housing the Homeless',
 '1 Hour Sleep Clinical Hypnosis Healing Sleep Music ',
 'When Covid Hit Nursing Homes, Part 1: ‘My Mother Died Alone’',
 'When Covid Hit Nursing Homes, Part 2: ‘They’re Not Giving Us an Ending’',
 '#1593 - Dr. Carl Hart',
 'MYSTERIOUS DEATH OF: Holly Bartlett',
 'Hour 2: Heat Homer Detonation',
 'How Blue Holes Work',
 'Death of a Hometown Hero',
 '7X NBA Champion Robert Horry, Coach K Losing His Program And Guys On Chicks',
 'Hell! Hell! Hell!',
 '#1586 - Tony Hinchcliffe',
 'Brené with Jennifer Rudolph Walsh and Ashley C. Ford on Hungry Hearts',
 'How Housing Discrimination Works',
 'MURDERED: Harvey and Jeannette Crewe',
 'I HAVE A BOYFRIEND',
 'Relaxing Rain Sounds for Deep Sleep | 2 Hours',
 'JRE MMA Show #103 with Max Holloway',
 'How Hydropower Works ',
 'Troopz Talking Soccer, This Is March, CBB And Drew Brees is Holding New Orleans Hostage',
 'Ask Him About His Credit Score (w/ Tiffany Haddish)',

In [23]:
getRecommendation('A Broken System for Housing the Homeless', 10)

The 10 most recommended episodes to A Broken System for Housing the Homeless are:

1 355 ‘Something Terrible Has Happened’ 
 https://open.spotify.com/episode/6UrrhuADN6aCM1ZrGLND4L

2 1168 Should Facebook Be Broken Up? 
 https://open.spotify.com/episode/7r6nom8BQxMipGU7DsfS3W

3 1889 ‘The Skunk at the Picnic’: Dr. Anthony Fauci on Working for Trump 
 https://open.spotify.com/episode/5leZDndsjs5vkqIpyQhR22

4 96 How They Stormed Congress 
 https://open.spotify.com/episode/6ehLGSSQ7IIiTiwg8zIpWm

5 2206 ‘Rankly Unfit’: The View From a Republican Who Voted to Impeach 
 https://open.spotify.com/episode/2RO4zuWX9a9vIBR768q6C5

6 3296 They Don't Care About You w/ Michael Malice 
 https://open.spotify.com/episode/74DGTrfKfZ1uOXr8H4ffZI

7 1070 Tarana Burke and Brené on Being Heard and Seen 
 https://open.spotify.com/episode/1zLUgXuBxyJJW6s87oZuKt

8 3022 Episode 193: Son of Sam Part I - Kill For My Master 
 https://open.spotify.com/episode/0oHR18K1E5YupAPN0KTmDt

9 2229 The Truth ft. Paige Lo

In [24]:
getRecommendation('Healthy Relationships', 10)

The 10 most recommended episodes to Healthy Relationships are:

1 3048 Take On Me 
 https://open.spotify.com/episode/7CeaT31t2w5fs9CoS3YMnx

2 1031 Basic Isn’t Bad 
 https://open.spotify.com/episode/13ZduA2jS5eEkJuQHXdAMs

3 436 Hobbies 
 https://open.spotify.com/episode/4NjVo4WHtym63Sr8VLDvOS

4 2953 Olivia O'Brien: Topping Music Charts, TikTok Stardom, & Dating Toxic Men 
 https://open.spotify.com/episode/6WREfroMWVJBQnh9Rx67Hf

5 1139 Life Burnout 
 https://open.spotify.com/episode/5CroFt0EQ3sPdkYfqfxa2z

6 2432 You Can’t Please Everyone: Friends, Relationships & Parents 
 https://open.spotify.com/episode/4wAe9W7pJyyfYlCYd7TwOw

7 2367 Food, Family, Friends & Futility 
 https://open.spotify.com/episode/7DlfyMXbxGqkdodZB6ww2m

8 1966 Snacks Minute for Monday, February 15th 
 https://open.spotify.com/episode/7JN6c4ZVKTNdxpyOs07EeO

9 631 Going On His First Date 
 https://open.spotify.com/episode/3axsrxv6nHSjwjOUEFIeHb

10 1042 Madison Beer On Our Relationship 
 https://open.spotify.co

In [25]:
getRecommendation('How to Decide', 10)

The 10 most recommended episodes to How to Decide are:

1 1814 #116 - a guide to being offensive 
 https://open.spotify.com/episode/69QyUQGiowlS8ryl74zD46

2 404 How To Make A Hard Decision 
 https://open.spotify.com/episode/471NtMPvGruOqZNpm06AXI

3 1207 Is It Better to Know? 
 https://open.spotify.com/episode/5QzA9vf5d5Is3iBhZ4NtfC

4 1962 F*** The Spark: The Science Of Finding A Partner 
 https://open.spotify.com/episode/3lSEKN8Q8aDfrgyBM8XzJq

5 3062 Master Your Days With This Mindset 
 https://open.spotify.com/episode/1X9NKs7dKqaBElomk9MwPv

6 381 Halloween, in an old house 
 https://open.spotify.com/episode/4G612HmhRS79xsic2M43oM

7 208 How to Get a Handle on the Voice in Your Head 
 https://open.spotify.com/episode/6YVMN2aCA2WsbUaPK1qSmd

8 599 Coffee On The Stoop or how to have a better day 
 https://open.spotify.com/episode/2JuaGMEtIyfKwqXYDkuD8r

9 613 Life is Hard Now or Hard Later 
 https://open.spotify.com/episode/5vsEVMokON2c0wgeqbkLWf

10 2231 Friendsgiving 
 https://ope